In [1]:
!pip install -Uq upgini catboost



In [2]:
from os.path import exists
import pandas as pd

df_path = "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=10000 , random_state=0)
df["store"] = df["store"].astype(str) # conv to str as we want the numbered column to be perceived like it has categories, not continuous vals to perform operations on
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"]) # interpret as date, not standard text

df.sort_values("date", inplace = True)
df.reset_index(inplace = True, drop = True) # discard the old index (that is now random) and replace it w a new sequenced one
df.head()

,date,store,item,sales
0,2013-01-01,5,24,26
1,2013-01-02,2,7,47
2,2013-01-03,6,21,23
3,2013-01-03,6,1,11
4,2013-01-05,9,37,12


In [3]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [4]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [5]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

api_key = "zwOZikBNSHXsTU69sc9j5piRiHNuSadQ22oHuBZS46OGp"

enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE,
    },
    CV = CVType.time_series
)

enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

<IPython.core.display.Javascript object>

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



<IPython.core.display.Javascript object>

Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=a78f8859-acf6-4a23-8ab3-b1071f54a545
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



store,0.3717,0.0199,100.0000,"10, 9, 1",,,
f_autofe_div_1967b510aa,0.0692,0.0713,100.0000,"0.9963, -35.3381, 0.233",Upgini,AutoFE: features from Calendar data,Daily
f_events_date_week_sin1_847b5db1,0.0687,0.0060,100.0000,"-0.7818, -0.4339, 0.9749",Upgini,Calendar data,Daily
f_autofe_mul_47f0121f33,0.0513,0.0843,100.0000,"0.197, 0.3616, -0.2263",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_div_996c59f173,0.0499,0.0738,100.0000,"0.7475, 0.538, 0.901",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_groupbythenrank_110e5b8df1,0.0416,0.0988,100.0000,"0.9234, 0.4123, 0.9245","Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",Daily
f_autofe_div_fc56b3ac6b,0.0303,0.0908,100.0000,"-5.2416, 0.6894, 0.4288",Upgini,"AutoFE: features from Calendar data,LLM with external data augmentation",Daily
f_events_date_year_sin2_59955ffd,0.0251,0.1347,100.0000,"-0.8958, 0.6681, 0.9894",Upgini,Calendar data,Daily
f_autofe_mul_04645a3381,0.0166,0.0662,100.0000,"-0.8366, -0.1706, -0.9113",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_mul_fc6603bc0c,0.0146,0.0545,100.0000,"-0.1807, -0.3474, -0.1163",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_groupbythenrank_67e74781ad,0.0096,0.0892,100.0000,"0.7109, 0.8974, 0.8468","Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",Daily


Upgini,AutoFE: features from Calendar data,0.2016,5
Upgini,Calendar data,0.0938,2
"Upgini,Training dataset","AutoFE: feature from Calendar data, grouped by feature from training dataset",0.0466,2
Upgini,"AutoFE: features from Calendar data,LLM with external data augmentation",0.0303,1
"Upgini,Training dataset","AutoFE: feature from Markets data, grouped by feature from training dataset",0.0116,2


Calendar data,f_autofe_div_1967b510aa,f_events_date_year_cos9_f85b1052,f_events_date_year_sin8_6be64013,/
Calendar data,f_autofe_mul_47f0121f33,f_events_date_year_cos8_d2a83ca9,f_events_date_year_sin7_16d9d7c9,*
Calendar data,f_autofe_div_996c59f173,f_events_date_year_cos10_09a4d418,f_events_date_year_cos3_0570af1d,/
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_110e5b8df1,f_events_date_week_cos1_f6a8c1fc,store_824d80,GroupByThenRank
"Calendar data,LLM with external data augmentation",f_autofe_div_fc56b3ac6b,f_events_date_year_cos5_59183b20,datetime_day_in_quarter_cos_eeb97a,/
Calendar data,f_autofe_mul_04645a3381,f_events_date_year_sin2_59955ffd,f_events_date_year_sin5_458f51f9,*
Calendar data,f_autofe_mul_fc6603bc0c,f_events_date_year_cos9_f85b1052,f_events_date_year_sin6_af4ea895,*
"Markets data, grouped by feature from training dataset",f_autofe_groupbythenrank_67e74781ad,f_financial_date_usd_daydiff_8bcdeec4,store_824d80,GroupByThenRank
"Calendar data, grouped by feature from training dataset",f_autofe_groupbythenrank_6289e10492,f_events_date_week_cos2_b0a07cfc,store_824d80,GroupByThenRank
"Markets data, grouped by feature from training dataset",f_autofe_groupbythenrank_ff14a787af,f_financial_date_nasdaq_gap_83732bb1,store_824d80,GroupByThenRank


In [6]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline MAPE,Enriched MAPE,"Uplift, abs","Uplift, %"
0,Train,949,51.1106,0.644 ± 0.112,0.596 ± 0.101,0.048,7.5%
1,Eval 1,251,63.0478,0.489 ± 0.009,0.459 ± 0.011,0.030,6.1%


In [8]:
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)

if enriched_train_features is not None:
    enriched_train_features.head()
else:
    print("Warning: enriched_train_features is None, likely due to upgini user limits.")
    print("Please check the output of the previous cell for 'Unregistered-user limit' messages.")

[============================================================] 100% Finished

Unregistered-user limit: 20 rows remaining; you requested 949.


Button(description='Get an API KEY', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Register', _do…

[============================================================] 100% Finished

Unregistered-user limit: 20 rows remaining; you requested 251.


Button(description='Get an API KEY', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Register', _do…

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

In [ ]:
set(enriched_train_features.columns) == set(enriched_test_features.columns)
